In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

In [3]:
import os
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
port = 5000

In [4]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                     TextMessage(text=event.message.text)]
            )
        )

@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/May/2024 09:19:10] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}


127.0.0.1 - - [09/May/2024 09:19:15] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}


...

BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"sticker","id":"507369155876618657","quoteToken":"oJnoIAk2xOv-YynLcvm4LvtvnEqafxLneGhMTxI0BvVm5PVlodesWxtPS2JWR5IHtscWnle7S-wcLCMIOSQsfy6v9n33RX-wLVUKaFCsqZOkDyNZfrLcUgQ6V34J0SH19S83t9eizeQzdiOG_q4xZw","stickerId":"693110","packageId":"2000017","stickerResourceType":"STATIC"},"webhookEventId":"01HXEAWWNENYTB7SVYPSPCGANE","deliveryContext":{"isRedelivery":false},"timestamp":1715246887092,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"92b43e3afa0f477fa4196b9eec87ede4","mode":"active"}]}


127.0.0.1 - - [09/May/2024 09:30:33] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"sticker","id":"507369399800299542","quoteToken":"5vLoe_LNlMfaBvh3hQGgJuhru4J5-dnYNRDzHRtGLliU3ju4eI5fjVc5IO2h1HXgYyPn27ClJqiC71qtIKm502f8FHIse9rbsiekYAz4Mfv5iJvyYK1oxI2FbTyTRpQG4y37MiKQb_e9x9cEyNs2mQ","stickerId":"141","packageId":"2","stickerResourceType":"STATIC","keywords":["summer","OK","Bored","circus","ball","Fooling around","summer vacation","Gotcha","Affirmative","ocean","swim","balance","play"]},"webhookEventId":"01HXEB1AMFFNT5EFNADYQEGEES","deliveryContext":{"isRedelivery":false},"timestamp":1715247032612,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"74ddb36ce55542b6af37abc4b62d881e","mode":"active"}]}


127.0.0.1 - - [09/May/2024 10:44:49] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"507376876063293915","quoteToken":"dcAIP2kuakRaFW-aLTxxKLAI7TRIbFept0BcP3gz2olRzdbVO9WFeQiAW6nDIaLfq3-xdhqN3KopVw1IKFxeW8lUrjWmCIiiJ8G--vs0wPpqEsqV9DXymZFJKpt7pac6-14UX3aQtsXkZAG2TFp3Kg","text":"分"},"webhookEventId":"01HXEF99ZV87YSSR9F9HXSES9P","deliveryContext":{"isRedelivery":false},"timestamp":1715251488701,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"d5698d85a6d846bb968b86b76a9e109b","mode":"active"}]}
